In [ ]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#改配色方案
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)
mpl.rcParams['font.family'] = 'Times New Roman'

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

In [ ]:
data=pd.read_csv('data.csv')
item=3

data['time']=pd.to_datetime(data['time'],format = '%Y-%m-%d %H:%M:%S')

turbulence = np.zeros((len(data), 1))

speed_for_turbulence = np.array(data['speed(m/s)_%s'%item])

for i in range(5,len(data)-1):
  tmp = speed_for_turbulence[(i-5):(i+1)]
  turbulence[i][0] = np.std(tmp)/(np.mean(tmp)+0.001)

turbulence[turbulence>1] = 1
data['turbulence_%s'%item]=turbulence

angle=np.array(data['angle(°)_%s'%item])
cos_angle=[np.cos(np.pi*(i/180)) for i in angle]
data['angle(°)_%s'%item]=cos_angle

norm_data_original=data[data['state_%s'%item]==10]
norm_data1=norm_data_original[norm_data_original['active_value_%s'%item]==100]
norm_data2=norm_data_original[norm_data_original['active_value_%s'%item]==0]

norm_data=pd.concat([norm_data1,norm_data2])

ab_data=data[data['state_%s'%item]!=10]
limit_data=pd.concat([norm_data_original,norm_data],axis =0).drop_duplicates(keep=False)
limit_data=pd.concat([limit_data,ab_data])

speed_all=np.array(data['speed(m/s)_%s'%item])
power_all=np.array(data['power(kW)_%s'%item])
temperature_all=np.array(data['temperature(℃)_%s'%item])
angle_all=np.array(data['angle(°)_%s'%item])
turbulence_all=np.array(data['turbulence_%s'%item])


speed=np.array(norm_data['speed(m/s)_%s'%item])
power=np.array(norm_data['power(kW)_%s'%item])
temperature=np.array(norm_data['temperature(℃)_%s'%item])
angle=np.array(norm_data['angle(°)_%s'%item])
turbulence=np.array(norm_data['turbulence_%s'%item])

speed_index=norm_data['speed(m/s)_%s'%item].index 

#generate dataset
speed_data=[[] for i in range(120)]
power_data=[[] for i in range(120)]
tem_data=[[] for i in range(120)]
angle_data=[[] for i in range(120)]
turbulence_data=[[] for i in range(120)]

speed_index=norm_data['speed(m/s)_%s'%item].index
for i in speed_index:
    for j in range(120):
        speed_data[j].append(speed_all[i-j])

power_index=norm_data['power(kW)_%s'%item].index

for i in power_index:
    for j in range(120):
        power_data[j].append(power_all[i-j])
    
tem_index=norm_data['temperature(℃)_%s'%item].index

for i in tem_index:
    for j in range(120):
        tem_data[j].append(temperature_all[i-j])

angle_index=norm_data['angle(°)_%s'%item].index

for i in angle_index:
    for j in range(120):
        angle_data[j].append(angle_all[i-j])

turbulence_index=norm_data['turbulence_%s'%item].index

for i in turbulence_index:
    for j in range(120):
        turbulence_data[j].append(turbulence_all[i-j])
        
df=pd.DataFrame()
for j in range(120):
    df['speed_%s'%j]=speed_data[j]
    
df_power=pd.DataFrame()
for j in range(120):
    df_power['power_%s'%j]=power_data[j]

df_tem=pd.DataFrame()
for j in range(120):
    df_tem['tem_%s'%j]=tem_data[j]

df_angle=pd.DataFrame()
for j in range(120):
    df_angle['angle_%s'%j]=angle_data[j]

df_turbulence=pd.DataFrame()
for j in range(120):
    df_turbulence['turbulence_%s'%j]=turbulence_data[j]

speed_df=pd.DataFrame()
speed_df['speed0']=df['speed_0']
speed_df['power1']=df_power['power_1']
speed_df['tem_0']=df_tem['tem_0']
speed_df['angle_0']=df_angle['angle_0']
speed_df['turbulence_0']=df_turbulence['turbulence_0']

speed_df['speed1']=df['speed_1']
speed_df['power2']=df_power['power_2']
speed_df['tem_1']=df_tem['tem_1']
speed_df['angle_1']=df_angle['angle_1']
speed_df['turbulence_1']=df_turbulence['turbulence_1']

speed_df['speed2']=df['speed_2']
speed_df['power3']=df_power['power_3']
speed_df['tem_2']=df_tem['tem_2']
speed_df['angle_2']=df_angle['angle_2']
speed_df['turbulence_2']=df_turbulence['turbulence_2']

speed_df['speed3']=df['speed_3']
speed_df['power4']=df_power['power_4']
speed_df['tem_3']=df_tem['tem_3']
speed_df['angle_3']=df_angle['angle_3']
speed_df['turbulence_3']=df_turbulence['turbulence_3']

speed_df['turbulence_4']=df_turbulence['turbulence_4']

'\n#进行u-sd标准化\ncol_name=speed_df.columns\nfor item in col_name:\n    tmp=np.array(speed_df[item])\n    tmp_1=(tmp-np.mean(tmp))/np.std(tmp)\n    speed_df[item]=tmp_1\n\npower=(power-np.min(power))/(np.max(power)-np.min(power))\n'

In [ ]:
speed_df=pd.DataFrame()
power=(power-np.min(power))/(np.max(power)-np.min(power))

for i in range(4):
  speed_df['power_%s'%(i+1)]=df_power['power_%s'%(i+1)]
  speed_df['speed_%s'%i]=df['speed_%s'%i]
  speed_df['angle_%s'%i]=df_angle['angle_%s'%i]
  speed_df['tem_%s'%i]=df_tem['tem_%s'%i]

In [ ]:
# u-sd standardization
col_name=speed_df.columns
for item in col_name:
    tmp=np.array(speed_df[item])
    tmp_1=(tmp-np.mean(tmp))/np.std(tmp)
    speed_df[item]=tmp_1

speed=np.array(speed_df)

X_train, X_test, y_train, y_test = train_test_split(speed,power, test_size=0.2,)
X_train_1=X_train
X_test_1=X_test
X_train = np.reshape(X_train, (X_train.shape[0], 4, 4))
X_test = np.reshape(X_test, (X_test.shape[0], 4, 4))

In [ ]:
#LossHistory class
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # # acc
        # plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'dodgerblue', label='train loss')
        if loss_type == 'epoch':
            # # val_acc
            # plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'darkorange', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('loss')
        plt.legend(loc="upper right")
        plt.savefig('loss{0}'.format(loss_type))

    def acc_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'dodgerblue', label='train acc')
        # loss
        # plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'darkorange', label='val acc')
            # val_loss
            # plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc')
        plt.legend(loc="upper right")
        plt.savefig('acc{0}'.format(loss_type))

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [ ]:
TIME_STEPS=4
INPUT_DIM=4
SINGLE_ATTENTION_VECTOR=False

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    print(input_dim)
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='attention')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def get_layer_output(model,layer_name,inputs):
    layer = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    layer_out = layer.predict(inputs)
    return np.mean(layer_out,axis=0)


def create_probablistic_attention_lstm_model():
  mean_inputs = tf.keras.Input((4,4))
  #attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(128, return_sequences=True,trainable=True)(mean_inputs)
  mean_x=LSTM(64, return_sequences=False,trainable=True)(mean_x)
  mean_x = Dense(64,trainable=True)(mean_x)
  mean_x = Dense(32,trainable=True)(mean_x)
  mean_x=Dense(1,trainable=True)(mean_x)

  sigma_x =LSTM(128, return_sequences=True,trainable=False)(mean_inputs)
  sigma_x=LSTM(64, return_sequences=False,trainable=False)(sigma_x)
  sigma_x = Dense(32,trainable=False)(sigma_x)
  sigma_x=Dense(1,trainable=False)(sigma_x)

  x = tf.concat([mean_x, sigma_x],1)
  # x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
  # x = layers.Dense(2, activation='swish', kernel_regularizer="l2")(x)
  outputs=tfp.layers.IndependentNormal(1)(x)

  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def create_probablistic_attention_lstm_model_mean():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(128, return_sequences=True,trainable=True)(attention_mul)
  mean_x=LSTM(64, return_sequences=False,trainable=True)(mean_x)
  mean_x = Dense(64,trainable=True)(mean_x)
  mean_x = Dense(32,trainable=True)(mean_x)
  mean_x=Dense(1,trainable=True)(mean_x)

  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model

def create_probablistic_attention_lstm_model_variance():
  variance_inputs = tf.keras.Input((4,4),name='main_input')
  attention_mul = attention_3d_block(variance_inputs)
  variance_x = LSTM(128, return_sequences=True,)(attention_mul)
  variance_x=LSTM(64, return_sequences=False)(variance_x)
  variance_x = Dense(64,)(variance_x)
  variance_x = Dense(32)(variance_x)
  variance_x=Dense(1,)(variance_x)

  point_inputs = tf.keras.Input((1,),name='point_predict_input')
  x = keras.layers.concatenate([point_inputs, variance_x])

  outputs=tfp.layers.IndependentNormal(1)(x)

  model = keras.Model(inputs=[variance_inputs,point_inputs], outputs=outputs)
  return model


def BNN():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True,)(attention_mul)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x = Dense(16,)(mean_x)
  #mean_x=tfp.layers.DenseVariational(units=32,make_prior_fn=prior,make_posterior_fn=posterior,activation="sigmoid",)(mean_inputs)
  #mean_x=tfp.layers.DenseVariational(units=16,make_prior_fn=prior,make_posterior_fn=posterior,activation="sigmoid",)(mean_x)
  mean_x=Dense(2)(mean_x)
  outputs = tfp.layers.IndependentNormal(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def create_probablistic_attention_lstm_model_myown():
  mean_inputs = tf.keras.Input((4,4))
  #attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(128, return_sequences=True,trainable=True)(mean_inputs)
  mean_x=LSTM(64, return_sequences=False,trainable=True)(mean_x)
  mean_x = Dense(64,trainable=True)(mean_x)
  mean_x = Dense(32,trainable=True)(mean_x)
  mean_x=Dense(1,trainable=True)(mean_x)

  sigma_x =LSTM(128, return_sequences=True,trainable=False)(mean_inputs)
  sigma_x=LSTM(64, return_sequences=False,trainable=False)(sigma_x)
  sigma_x = Dense(32,trainable=False)(sigma_x)
  sigma_x=Dense(1,trainable=False)(sigma_x)

  outputs =layers.Concatenate(axis=1)([mean_x, sigma_x])

  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def create_probablistic_attention_lstm_model_2():
  mean_inputs = tf.keras.Input((4,4))
  #attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(128, return_sequences=True)(mean_inputs)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(64)(mean_x)
  mean_x = Dense(64)(mean_x)
  mean_x=Dense(2)(mean_x)

  outputs=tfp.layers.IndependentNormal(1)(mean_x)

  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def BNN_studentT():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True,)(attention_mul)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x = Dense(16,)(mean_x)

  mean_x=Dense(2)(mean_x)
  #outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.StudentT(df=100, loc=t[..., 0], scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  #outputs = tfp.layers.IndependentNormal(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_studentT():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True,)(attention_mul)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x = Dense(16,)(mean_x)

  mean_x=Dense(2)(mean_x)
  #outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.StudentT(df=100, loc=t[..., 0], scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  #outputs = tfp.layers.IndependentNormal(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_gamma():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True,)(attention_mul)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x = Dense(16,)(mean_x)

  mean_x=Dense(2)(mean_x)
  #outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Gamma(concentration=tf.abs(t[..., 0]),rate=tf.abs(t[..., 1])))(mean_x)
  #outputs = tfp.layers.IndependentNormal(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_laplace():
  mean_inputs = tf.keras.Input((4,4))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True,)(attention_mul)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x = Dense(16,)(mean_x)

  mean_x=Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Laplace(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

In [ ]:
import keras.backend as K
import math as m
def negative_loglikelihood(targets,estimated_distribution):
  #0.5*K.log(tf.constant(2*m.pi,dtype=tf.float32))+0.5*K.log(preds[1]*preds[1])+((targets-preds[0])**2)/(2*preds[1]*preds[1])
  return -estimated_distribution.log_prob(targets)

### Train and Predict

In [ ]:
X_train = np.load('distribution_analysis/X_train.npy', allow_pickle=True)
X_test = np.load('distribution_analysis/X_test.npy', allow_pickle=True)
y_train = np.load('distribution_analysis/y_train.npy', allow_pickle=True)
y_test = np.load('distribution_analysis/y_test.npy', allow_pickle=True)

In [ ]:
model=BNN_gamma()

learning_rate=0.001
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),loss=negative_loglikelihood,metrics=[keras.metrics.RootMeanSquaredError()],)
model.fit(X_train,y_train,batch_size=64, epochs=100,validation_split=0.15)

prediction_distribution = model(X_test)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

Distribution type discussion

In [ ]:
def find_nearest(a, a0):
  idx = np.abs(a - a0).argmin()
  return idx

def get_quantile(distribution):
  raw=np.arange(0,1,0.001)
  cdf_list=[]
  for value in raw:
    cdf_list.append(distribution.cdf(value).numpy()[0])

  quantile_list=[]
  for item in np.arange(0.1,1,0.1):
    idx=find_nearest(cdf_list,item)
    quantile_list.append(raw[idx])

  return quantile_list

In [ ]:
from tqdm import tqdm

quantile_prediction=[]
for i in tqdm(range(len(X_test))):
  prediction_distribution = model(X_test[i].reshape(1,4,4))
  quantile=get_quantile(prediction_distribution)
  quantile_prediction.append(quantile)

Result measure

In [ ]:
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score
from sklearn import metrics

r_2=r2_score(prediction_mean,y_test)
rmse = np.sqrt(MSE(prediction_mean,y_test))
mae = MAE(prediction_mean,y_test)
mape_ = metrics.mean_absolute_percentage_error(prediction_mean,y_test)
mse=MSE(prediction_mean,y_test)

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.048610748466574434
mae:0.03260880448551605
r2:0.9688960098152606
mape:0.12762777495481495


In [ ]:
#CRPS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

raw_point=y_test
pred_point=prediction_mean
std_data=prediction_stdv
crps_sum=0
mcrps=[]
for j in range(len(raw_point)):
    crps_sum+=std_data[j]*(((raw_point[j]-pred_point[j])/std_data[j])*(2*st.norm.cdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1)+
                               2*st.norm.pdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1/np.sqrt(3.1415926))
mcrps.append(crps_sum/len(raw_point))

In [ ]:
mcrps

[0.0240446764900032]

In [ ]:
#MSC
#msc
sc_sum=0
MSC=[]
for j in range(len(raw_point)):
    # 均值为pred_point，标准差为pred_residual的正态分布在累积概率为0.5时对应的函数值
    cdf_point=st.norm.ppf([0.10,0.20,0.30,0.40,0.5,0.60,0.70,0.80,0.90,], loc=pred_point[j], scale=std_data[j])
    tmp=raw_point[j]<=cdf_point
    tmp=tmp+0
    sc_sum+=np.sum((tmp-[0.10,0.20,0.30,0.40,0.5,0.60,0.70,0.80,0.90,])*(raw_point[j]-cdf_point))
MSC.append(sc_sum/len(raw_point))

In [ ]:
MSC

[-0.11802319056750858]

Attention score

In [ ]:
att = get_layer_output(model,'attention_vec',X_test)
print(att.flatten())
x = np.array(speed_df.columns)

sns.barplot(att.flatten(),x)
print(np.sum(att))

## metric

In [ ]:
pred_result=pd.read_csv('result/pa_lstm_result_offline.csv')

In [ ]:
pred_result.head()

sharpness and reliability

In [ ]:
z_value=[2.58,1.96,1.65,1.29,1.04,0.85,0.68,0.53,0.39,0.26,0.13]
confidence=[0.99,0.95,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
shapness=[[] for i in range(11)]
reliability=[[] for i in range(11)]

In [ ]:
# intercal prediction interval
#shapness and reliability
raw_point=np.array(pred_result['raw_data'])
pred_point=np.array(pred_result['pred_mean'])
std_data=np.array(pred_result['pred_std'])

for j in range(11):
  upper_point=pred_point+z_value[j]*np.array(std_data)
  lower_point=pred_point-z_value[j]*np.array(std_data)
  shapness[j].append(np.mean(upper_point-lower_point))
  reliability[j].append(np.sum((raw_point<=upper_point)&(raw_point>=lower_point))/len(raw_point)-confidence[j])

In [ ]:
result_df=pd.DataFrame()
result_df['sharpness_pa_lstm']=np.array(shapness).flatten()
result_df['reliability_pa_lstm']=np.array(reliability).flatten()

In [ ]:
result_df.to_csv('result/sha_rel.csv',index=False)

skill score and crps

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

#mcrps
crps_sum=0
mcrps=[]
for j in range(len(raw_point)):
    # 均值为pred_point，标准差为pred_residual的正态分布在累积概率为0.5时对应的函数值
    crps_sum+=std_data[j]*(((raw_point[j]-pred_point[j])/std_data[j])*(2*st.norm.cdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1)+
                               2*st.norm.pdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1/np.sqrt(3.1415926))
mcrps.append(crps_sum/len(raw_point))

In [ ]:
#MSC
#msc
sc_sum=0
MSC=[]
for j in range(len(raw_point)):
    # 均值为pred_point，标准差为pred_residual的正态分布在累积概率为0.5时对应的函数值
    cdf_point=st.norm.ppf([0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.5,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95], loc=pred_point[j], scale=std_data[j])
    tmp=raw_point[j]<=cdf_point
    tmp=tmp+0
    sc_sum+=np.sum((tmp-[0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.5,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95])*(raw_point[j]-cdf_point))
MSC.append(sc_sum/len(raw_point))

In [ ]:
MSC

[-0.430385673175324]

### Baseline

PPC

In [ ]:
!pip install fitter

In [ ]:
from fitter import Fitter

speed_df=pd.DataFrame()
power=(power-np.min(power))/(np.max(power)-np.min(power))

speed_df['speed_0']=df['speed_0']
speed_df['power']=power

#15m/s is the boundary
speed_gap=[round(2+0.1*i,2) for i in range(131)]

mean_list=[]
std_list=[]
for item in speed_gap:
  fit_power=speed_df[(speed_df['speed_0']>=item)&(speed_df['speed_0']<(item+0.1))]['power']
  fit_power=np.array(fit_power)
  f = Fitter(fit_power, distributions=['norm',])
  f.fit()
  norm_mean=f.fitted_param['norm'][0]
  norm_std=f.fitted_param['norm'][1]
  mean_list.append(norm_mean)
  std_list.append(norm_std)

norm_df=pd.DataFrame()
norm_df['speed_range']=speed_gap
norm_df['mean']=mean_list
norm_df['std']=std_list

###The previous code generates the mean and variance for each speed

###The latter code generates the mean and variance for each speed

#norm_df.to_csv('ppc_offline.csv',index=False)
X_train, X_test, y_train, y_test = train_test_split(np.array(speed_df['speed_0']),np.array(speed_df['power']),test_size=0.2,)

X_test_new=[]
for item in X_test:
  if item<=15:
    X_test_new.append(round(item,1))
  else:
    X_test_new.append(15)

#forecast
pred_mean=[]
pred_std=[]
for i in range(len(X_test_new)):
  tmp_mean=norm_df[norm_df['speed_range']==X_test_new[i]]['mean'].values[0]
  tmp_std=norm_df[norm_df['speed_range']==X_test_new[i]]['std'].values[0]
  pred_mean.append(tmp_mean)
  pred_std.append(tmp_std)

pred_mean=np.array(pred_mean)
pred_std=np.array(pred_std)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score
from sklearn import metrics

r_2=r2_score(pred_mean,y_test)
rmse = np.sqrt(MSE(pred_mean,y_test))
mae = MAE(pred_mean,y_test)
mape_ = metrics.mean_absolute_percentage_error(pred_mean,y_test)
mse=MSE(pred_mean,y_test)

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.048236169569808175
mae:0.03219477467365577
r2:0.9702071397766286
mape:0.1465063948012958


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

#mcrps
raw_point=y_test
pred_point=pred_mean
std_data=pred_std
crps_sum=0
mcrps=[]
for j in range(len(raw_point)):
    crps_sum+=std_data[j]*(((raw_point[j]-pred_point[j])/std_data[j])*(2*st.norm.cdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1)+
                               2*st.norm.pdf((raw_point[j]-pred_point[j])/std_data[j],0,1)-1/np.sqrt(3.1415926))
mcrps.append(crps_sum/len(raw_point))

In [ ]:
#MSC
#msc
sc_sum=0
MSC=[]
for j in range(len(raw_point)):
    cdf_point=st.norm.ppf([0.10,0.20,0.30,0.40,0.5,0.60,0.70,0.80,0.90,], loc=pred_point[j], scale=std_data[j])
    tmp=raw_point[j]<=cdf_point
    tmp=tmp+0
    sc_sum+=np.sum((tmp-[0.10,0.20,0.30,0.40,0.5,0.60,0.70,0.80,0.90,])*(raw_point[j]-cdf_point))
MSC.append(sc_sum/len(raw_point))

KDE

In [ ]:
import numpy as np

def get_kde(x,data_array,bandwidth):
    def gauss(x):
        import math
        return (1/math.sqrt(2*math.pi))*math.exp(-0.5*(x**2))
    N=len(data_array)
    res=0
    if len(data_array)==0:
        return 0
    for i in range(len(data_array)):
        res += gauss((x-data_array[i])/bandwidth)
    res /= (N*bandwidth)
    return res

def kde_y(input_array,c):
    bandwidth=c*np.std(input_array)*(len(input_array)**(-1/5))
    x_array=np.linspace(min(input_array),max(input_array),100)
    y_array=[get_kde(x_array[i],input_array,bandwidth) for i in range(x_array.shape[0])]
    return x_array,y_array

def quantile_ind(y_array,x_array,a):
    s1 = 0
    index = 0
    for i in y_array:
        s1+=i*(x_array[1]-x_array[0])
        index+=1
        if s1>a:
            break
    return index-1

def interpolation(x_array,y_array,index,a):

    d = x_array[1]-x_array[0]
    if index !=0:
       x = (a-np.sum(y_array[:index])*d)/(y_array[index]*d)*(x_array[index]-x_array[index-1])+x_array[index-1]
       return x
    else:
       #x = a/(y_array[index]*d)*x_array[index]
       x = x_array[0]
       return x

In [ ]:
speed_df=pd.DataFrame()
power=(power-np.min(power))/(np.max(power)-np.min(power))

speed_df['speed_0']=df['speed_0']
speed_df['power']=power

#15m/s is the boundary
speed_gap=[round(2+0.1*i,2) for i in range(131)]


y10=[]
y20=[]
y30=[]
y40=[]
y50=[]
y60=[]
y70=[]
y80=[]
y90=[]

ind10=[]
ind20=[]
ind30=[]
ind40=[]
ind50=[]
ind60=[]
ind70=[]
ind80=[]
ind90=[]

for item in speed_gap:
  fit_power=speed_df[(speed_df['speed_0']>=item)&(speed_df['speed_0']<(item+0.1))]['power']
  fit_power=np.array(fit_power)
  x_array, y_array = kde_y(fit_power,c=1.06)
  in10 = quantile_ind(y_array,x_array,0.1)
  in20 = quantile_ind(y_array,x_array,0.2)
  in30 = quantile_ind(y_array,x_array,0.3)
  in40 = quantile_ind(y_array,x_array,0.4)
  in50 = quantile_ind(y_array,x_array,0.5)
  in60 = quantile_ind(y_array,x_array,0.6)
  in70 = quantile_ind(y_array,x_array,0.7)
  in80 = quantile_ind(y_array,x_array,0.8)
  in90 = quantile_ind(y_array,x_array,0.9)

  ind10.append(in10)
  ind20.append(in20)
  ind30.append(in30)
  ind40.append(in40)
  ind50.append(in50)
  ind60.append(in60)
  ind70.append(in70)
  ind80.append(in80)
  ind90.append(in90)
  y10.append(interpolation(x_array,y_array,in10,0.1))
  y20.append(interpolation(x_array,y_array,in20,0.2))
  y30.append(interpolation(x_array,y_array,in30,0.3))
  y40.append(interpolation(x_array,y_array,in40,0.4))
  y50.append(interpolation(x_array,y_array,in50,0.5))
  y60.append(interpolation(x_array,y_array,in60,0.6))
  y70.append(interpolation(x_array,y_array,in70,0.7))
  y80.append(interpolation(x_array,y_array,in80,0.8))
  y90.append(interpolation(x_array,y_array,in90,0.9))

In [ ]:
norm_df=pd.DataFrame()
norm_df['speed_range']=speed_gap
norm_df['q10']=y10
norm_df['q20']=y20
norm_df['q30']=y30
norm_df['q40']=y40
norm_df['q50']=y50
norm_df['q60']=y60
norm_df['q70']=y70
norm_df['q80']=y80
norm_df['q90']=y90

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(speed_df['speed_0']),np.array(speed_df['power']),test_size=0.2,)

X_test_new=[]
for item in X_test:
  if item<=15:
    X_test_new.append(round(item,1))
  else:
    X_test_new.append(15)

#预测
y10=[]
y20=[]
y30=[]
y40=[]
y50=[]
y60=[]
y70=[]
y80=[]
y90=[]

for i in range(len(X_test_new)):
  y10.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q10'].values[0])
  y20.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q20'].values[0])
  y30.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q30'].values[0])
  y40.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q40'].values[0])
  y50.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q50'].values[0])
  y60.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q60'].values[0])
  y70.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q70'].values[0])
  y80.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q80'].values[0])
  y90.append(norm_df[norm_df['speed_range']==X_test_new[i]]['q90'].values[0])

In [ ]:
pred_df=pd.DataFrame()
pred_df['q10']=y10
pred_df['q20']=y20
pred_df['q30']=y30
pred_df['q40']=y40
pred_df['q50']=y50
pred_df['q60']=y60
pred_df['q70']=y70
pred_df['q80']=y80
pred_df['q90']=y90

pred_df['raw']=y_test
pred_df.to_csv('result/KDE_inshore.csv',index=False)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score
from sklearn import metrics

prediction_mean=np.array(y50)
r_2=r2_score(prediction_mean,y_test)
rmse = np.sqrt(MSE(prediction_mean,y_test))
mae = MAE(prediction_mean,y_test)
mape_ = metrics.mean_absolute_percentage_error(prediction_mean,y_test)
mse=MSE(prediction_mean,y_test)

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.05501909424925353
mae:0.03746694257729789
r2:0.9607973362980033
mape:0.14529681322480595


In [ ]:
#MSC
sc_sum=0
MSC=[]
raw_point=y_test
test_pred=np.array(pred_df[pred_df.columns[0:-1]])
for j in range(len(raw_point)):
    # Function value of the normal distribution with mean pred_point and standard deviation pred_residual when the cumulative probability is 0.5
    cdf_point=test_pred[j]
    tmp=raw_point[j]<=cdf_point
    tmp=tmp+0
    sc_sum+=np.sum((tmp-[0.10,0.20,0.30,0.40,0.5,0.60,0.70,0.80,0.90])*(raw_point[j]-cdf_point))
MSC.append(sc_sum/len(raw_point))

In [ ]:
MSC

[-0.1318268087514801]